In [27]:
from PIL import Image
import pandas as pd
import numpy as np
from glob import glob
from os import path
import random 
import matplotlib.pyplot as plt 
import tensorflow
import imageio
from scipy import signal
from pathlib import Path


In [30]:
## IMPORTS ALL IMAGES WITHIN THE FOLDER "imager_dir" INTO TWO ARRAYS, ONE FOR GAUSS POINTS AND ONE FOR IMAGE CROPS ##
all_image_array=[]
all_image_array_gauss=[]

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf_pos'
joined_path = os.path.join(files_dir, images_dir)

for image_file in os.listdir(joined_path):
    image, date, cell_type, bf_fl, index, number_gauss  = image_file.split('_')
    joined_image_path = os.path.join(files_dir, images_dir, image_file)

    if 'gauss' in number_gauss:
        img_gauss = Image.open(joined_image_path)
        image_array_gauss = np.array(img_gauss)
        all_image_array_gauss.append(image_array_gauss)
    else:
        img = Image.open(joined_image_path)
        image_array = np.array(img)
        all_image_array.append(image_array)


In [34]:
## ANYTHING BELOW THIS POINT IS FROM MITOSPLITNET CODE "augmentation" ##
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import label2rgb
from skimage import segmentation

from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize

In [ ]:
def augImg(input_img, output_img, transform, **kwargs):
    input_mask = (input_img>0).astype(np.uint8)
    transformed = transform(image=input_img, image0=output_img)
    transformed['image'] = noise_transform(image=transformed['image'])['image']
    
    aug_input_img, aug_output_img, aug_labels = transformed['image']*transformed['mask0'], transformed['image0'], transformed['mask']
    
    aug_fission_coords = preprocessing.fissionCoords(aug_labels, aug_output_img)
    aug_output_img, aug_fission_props = preprocessing.prepareProc(aug_output_img, coords=aug_fission_coords, **kwargs)
    aug_labels = preprocessing.segmentFissions(aug_output_img, aug_fission_props)
    return aug_input_img.astype(np.uint8), aug_output_img, aug_labels
    

def augStack(input_data, output_data, transform, **kwargs):
    aug_input_data = np.zeros(input_data.shape, dtype=np.uint8)
    aug_output_data = np.zeros(output_data.shape, dtype=np.float32)
    
    for i in tqdm(range(input_data.shape[0]), total=input_data.shape[0]):
        aug_input_data[i], aug_output_data[i]= augImg(input_data[i], output_data[i], labels[i], transform, noise_transform, **kwargs)
    return aug_input_data, aug_output_data

In [36]:
img_size = 256
transform = Compose([RandomRotate90(p=0.5), HorizontalFlip(p=0.5), Flip(p=0.5)])

aug_input_data, aug_output_data= augmentation.augStack(all_image_array, all_image_array_gauss,transform, sigma=8)

nb_img = all_image_array.shape[0]
title_size=20
width = 5

fig, axes = plt.subplots(nb_img, 3, figsize=(width*3, width*nb_img))
for frame in range(all_image_array.shape[0]):
  plotting.plot_merge(all_image_array[frame],  all_image_array_gauss[frame], title='', ax=axes[frame, 0])
  plotting.plot_merge(aug_input_data[frame], aug_output_data[frame], title='', ax=axes[frame, 1])
  axes[frame, 2].imshow(label2rgb(aug_labels[frame], bg_label=0))
  axes[frame, 2].axis('off')
  
axes[0, 0].set_title('Original', size=title_size, color='white')
axes[0, 1].set_title('Augmented output', size=title_size, color='white')
axes[0, 2].set_title('Augmented labels', size=title_size, color='white')

fig.tight_layout(pad=0)
fig.subplots_adjust(hspace=0.05, wspace=0)

plt.show()

TypeError: augStack() missing 2 required positional arguments: 'transform' and 'noise_transform'